Студент Жилкин Федор, 344
# Домашняя работа 4: "Численное решение интегрального уравнения Фредгольма 2-го рода."
**Задания:**

1. Метод механических квадратур  
2. Метод замены ядра на вырожденное

$u(x) - 5\int\limits_0^1 \frac{ycos(x)}{xy-2}u(y)dy= 2 + \frac{e^{5x}-1}{2x}$

In [62]:
import numpy as np
import pandas as pd 
import sympy

In [64]:
# Используем предел для вычисления функции, 
# т.к. функция может не иметь значения в некоторых точках.
def f(x):
    X = sympy.symbols('x')
    expr = 2+(sympy.exp(5*X)-1)/(2*X)
    res = sympy.limit(expr, X ,x)
    return float(res)

In [40]:
def K(x, y):
    return (y*np.cos(x))/(x*y-2)

In [7]:
x = [i/10 for i in range(1,11)]
f = [f(i) for i in x]

In [78]:
# Рекурсивно записываем полином Лежандра
def Legendre(n,x):
    x=np.array(x)
    if (n==0):
        return x*0+1.0
    elif (n==1):
        return x
    else:
        return ((2.0*n-1.0)*x*Legendre(n-1,x)-(n-1)*Legendre(n-2,x))/n

In [79]:
def DLegendre(n,x):
    x=np.array(x)
    if (n==0):
        return x*0
    elif (n==1):
        return x*0+1.0
    else:
        return (n/(x**2-1.0))*(x*Legendre(n,x)-Legendre(n-1,x))

In [80]:
# Корни полинома Лежандра по методу Ньютона-Рафсона
def LegendreRoots(polyorder,tolerance=1e-20):
    if polyorder<2:
        err=1 
    else:
        roots=[]
        for i in range(1,int(polyorder/2 +1)):
            x = np.cos(np.pi*(i-0.25)/(polyorder+0.5))
            error = 10*tolerance
            iters = 0
            while (error>tolerance) and (iters<1000):
                dx=-Legendre(polyorder,x)/DLegendre(polyorder,x)
                x=x+dx
                iters=iters+1
                error=abs(dx)
            roots.append(x)
        roots=np.array(roots)
        if polyorder%2==0:
            roots=np.concatenate( (-1.0*roots, roots[::-1]) )
        else:
            roots=np.concatenate( (-1.0*roots, [0.0], roots[::-1]) )
        err=0 
    return [roots, err]

In [48]:
# Веса квадратурной формулы
def GaussLegendreWeights(polyorder):
    W=[]
    [xis,err]=LegendreRoots(polyorder)
    if err==0:
        W=2.0/( (1.0-xis**2)*(DLegendre(polyorder,xis)**2) )
        err=0
    else:
        err=1 
    return [W, xis, err]


In [49]:
def GaussLegendreQuadrature(func, polyorder, a, b):
    [Ws,xs, err]= GaussLegendreWeights(polyorder)
    if err==0:
        ans=(b-a)*0.5*sum( Ws*func( (b-a)*0.5*xs+ (b+a)*0.5 ) )
    else: 
        err=1
        ans=None
    return [ans,err]

In [50]:
GaussLegendreWeights(5)

[array([0.23692689, 0.47862867, 0.56888889, 0.47862867, 0.23692689]),
 array([-0.90617985, -0.53846931,  0.        ,  0.53846931,  0.90617985]),
 0]

In [83]:
# Метод механических квадратур
def mechanical_quadrature(n, a, b):
    """
    Решение интегрального уравнения методом механических квадратур
    n -- кол-во точек разбиения отрезка [-1, 1] (кол-во корней мн-на Лежандра)
    """
    W, X, er = GaussLegendreWeights(n)
    if er == 1:
        print("Ошибка в подборе корней Лежандра")
        return None
    A = np.zeros((n, n))
    B = np.zeros(n)
    for i in range(0, n):
        for j in range(0, n):
            if j == i:
                beta = 1
            else:
                beta = 0
            A[i][j] = beta -5*W[j]*(b-a)*0.5*K((b-a)*0.5*X[i]+ (b+a)*0.5, (b-a)*0.5*X[j]+ (b+a)*0.5)
        B[i] = f(X[i])
    return A, B

In [85]:
A, B = mechanical_quadrature(5, 0, 1)

In [88]:
z = np.linalg.solve(A, B)
z

array([-7.21567496, -7.76556239, -6.99955296,  3.13731602, 41.55739404])

In [ ]:
sum()

## Метод механических квадратур

In [25]:
def define(a, b):
    return a**3/3 - b**3/3

In [32]:
define(-4,6)

-93.33333333333333